In [1]:
import autokeras as ak
from pandas import read_csv
import pandas as pd
from sklearn.model_selection import train_test_split
import joblib

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7551442473667585434
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 15561695822802581959
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 5825486136705515947
physical_device_desc: "device: XLA_GPU device"
]


In [3]:
import wandb
wandb.login()

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ········


wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Олежечка/.netrc


True

In [4]:
import wandb
from wandb.keras import WandbCallback

wandb.init(project="autokeras-test")

from tensordash.tensordash import Tensordash
histories = Tensordash(
    ModelName = 'AutoKeras_test',
    email = 'obezrukavnikov@gmail.com',
    password = 'olezhechkab'
)

callbacks=[WandbCallback(), histories]

wandb: Currently logged in as: olezhechkab (use `wandb login --relogin` to force relogin)
d:\gitprojects\machinelearning\auto_ml\auto_keras\lib\site-packages\IPython\html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


In [5]:
test = read_csv("NTI_data/test.csv")
train_target = read_csv("NTI_data/train_target.csv")
transactions_test = read_csv("NTI_data/transactions_test.csv")
transactions_train = read_csv("NTI_data/transactions_train.csv")

In [6]:
#Train
#Признаки, связанные с суммой покупки
agg_features=transactions_train.groupby('client_id')['amount_rur'].agg(['sum','mean','max']).reset_index()

#Количество в каждой категории
amount_of_purchase=transactions_train.groupby(['client_id','small_group'])['amount_rur'].count()
counts_train=amount_of_purchase.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
counts_train=counts_train.fillna(0)
counts_train.columns=['small_group_' + str(i) for i in counts_train.columns]

#Сумма в каждой категории
sum_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].sum()
sum_train=sum_of_group.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
sum_train=sum_train.fillna(0)
sum_train.columns=['small_group_sum_'+str(i) for i in sum_train.columns]

#Минимум в каждой категории
min_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].min()
min_train=sum_of_group.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
min_train=min_train.fillna(0)
min_train.columns=['small_group_min_'+str(i) for i in min_train.columns]

#Максимум в каждой категории
max_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].min()
max_train=sum_of_group.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
max_train=max_train.fillna(0)
max_train.columns=['small_group_max_'+str(i) for i in max_train.columns]

#Средняя сумма в каждой категории
avg_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].mean()
avg_train=avg_of_group.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
avg_train=avg_train.fillna(0)
avg_train.columns=['small_group_avg_'+str(i) for i in avg_train.columns]

#Соединим все в одну таблицу
counts_train=pd.merge(sum_train,counts_train,on='client_id')
counts_train=pd.merge(avg_train,counts_train,on='client_id')
counts_train=pd.merge(min_train,counts_train,on='client_id')
counts_train=pd.merge(max_train,counts_train,on='client_id')
train=pd.merge(train_target,agg_features,on='client_id')
train=pd.merge(train,counts_train.reset_index(),on='client_id')

In [7]:
train_size = 500
train_data = train.head(train_size)
test_data = train.tail(29000)

In [8]:
import time
start_time = time.time()

In [9]:
model_finder = ak.StructuredDataClassifier(
    overwrite=True,
    max_trials=50
)
X_train = train_data.drop('bins', 1)
y_train = train_data['bins']
X_test = test_data.drop('bins', 1)
y_test = test_data['bins']

In [10]:
model_finder.fit(X_train, y_train, callbacks=callbacks)

Epoch 1/1000
 9/13 [===================>..........] - ETA: 0s - loss: 1711.3423 - accuracy: 0.312 - ETA: 0s - loss: 1453.8744 - accuracy: 0.3507

wandb: ERROR Can't save model, h5py returned error: Save or restore weights that is not an instance of `tf.Variable` is not supported in h5, use `save_format='tf'` instead. Got a model or layer MultiCategoryEncoding with weights [<tensorflow.python.keras.engine.base_layer_utils.TrackableWeightHandler object at 0x000002514C51F340>, <tensorflow.python.keras.engine.base_layer_utils.TrackableWeightHandler object at 0x000002514C50CA30>, <tensorflow.python.keras.engine.base_layer_utils.TrackableWeightHandler object at 0x000002514C50C2E0>, <tensorflow.python.keras.engine.base_layer_utils.TrackableWeightHandler object at 0x0000025145FE1D00>, <tensorflow.python.keras.engine.base_layer_utils.TrackableWeightHandler object at 0x0000025145FE1760>, <tensorflow.python.keras.engine.base_layer_utils.TrackableWeightHandler object at 0x000002514759AD90>, <tensorflow.python.keras.engine.base_layer_utils.TrackableWeightHandler object at 0x000002514759A640>, <tensorflow.python.keras.engine.base_layer_utils.

13/13 [==============================] - 6s 499ms/step - loss: 1289.2690 - accuracy: 0.3413 - val_loss: 627.6175 - val_accuracy: 0.2857
Epoch 2/1000
13/13 [==============================] - ETA: 0s - loss: 1284.1387 - accuracy: 0.218 - ETA: 0s - loss: 1222.5199 - accuracy: 0.276 - ETA: 0s - loss: 881.2635 - accuracy: 0.302 - 1s 97ms/step - loss: 881.2635 - accuracy: 0.3029 - val_loss: 421.9388 - val_accuracy: 0.3333
Epoch 3/1000
13/13 [==============================] - ETA: 0s - loss: 429.6815 - accuracy: 0.37 - ETA: 0s - loss: 540.1618 - accuracy: 0.41 - ETA: 0s - loss: 436.7232 - accuracy: 0.37 - 1s 100ms/step - loss: 436.7232 - accuracy: 0.3702 - val_loss: 212.5238 - val_accuracy: 0.3452
Epoch 4/1000
13/13 [==============================] - ETA: 0s - loss: 284.6063 - accuracy: 0.37 - ETA: 0s - loss: 443.7138 - accuracy: 0.40 - 1s 94ms/step - loss: 299.4539 - accuracy: 0.3990 - val_loss: 75.0262 - val_accuracy: 0.4643
Epoch 5/1000
13/13 [==============================] - ETA: 0s - lo

Epoch 1/1000
13/13 [==============================] - ETA: 0s - loss: 3102.8418 - accuracy: 0.156 - ETA: 0s - loss: 1958.9353 - accuracy: 0.276 - ETA: 0s - loss: 1687.7604 - accuracy: 0.2933

wandb: ERROR Can't save model, h5py returned error: Save or restore weights that is not an instance of `tf.Variable` is not supported in h5, use `save_format='tf'` instead. Got a model or layer MultiCategoryEncoding with weights [<tensorflow.python.keras.engine.base_layer_utils.TrackableWeightHandler object at 0x000002513CD1CD30>, <tensorflow.python.keras.engine.base_layer_utils.TrackableWeightHandler object at 0x000002513CD1C5B0>, <tensorflow.python.keras.engine.base_layer_utils.TrackableWeightHandler object at 0x000002513CE2FDC0>, <tensorflow.python.keras.engine.base_layer_utils.TrackableWeightHandler object at 0x00000251471A8760>, <tensorflow.python.keras.engine.base_layer_utils.TrackableWeightHandler object at 0x00000251471A84C0>, <tensorflow.python.keras.engine.base_layer_utils.TrackableWeightHandler object at 0x000002513CD72D60>, <tensorflow.python.keras.engine.base_layer_utils.TrackableWeightHandler object at 0x000002513CD72970>, <tensorflow.python.keras.engine.base_layer_utils.

...

Epoch 1/1000
11/13 [========================>.....] - ETA: 0s - loss: 1.9565 - accuracy: 0.21 - ETA: 0s - loss: 1.8249 - accuracy: 0.23 - ETA: 0s - loss: 1.8390 - accuracy: 0.2727

wandb: ERROR Can't save model, h5py returned error: Save or restore weights that is not an instance of `tf.Variable` is not supported in h5, use `save_format='tf'` instead. Got a model or layer MultiCategoryEncoding with weights [<tensorflow.python.keras.engine.base_layer_utils.TrackableWeightHandler object at 0x00000251EEE5C370>, <tensorflow.python.keras.engine.base_layer_utils.TrackableWeightHandler object at 0x00000251EEE5CA90>, <tensorflow.python.keras.engine.base_layer_utils.TrackableWeightHandler object at 0x000002518E0D4700>, <tensorflow.python.keras.engine.base_layer_utils.TrackableWeightHandler object at 0x000002518E0D4160>, <tensorflow.python.keras.engine.base_layer_utils.TrackableWeightHandler object at 0x000002514CD3F790>, <tensorflow.python.keras.engine.base_layer_utils.TrackableWeightHandler object at 0x000002514CD3F370>, <tensorflow.python.keras.engine.base_layer_utils.TrackableWeightHandler object at 0x00000251A06F45B0>, <tensorflow.python.keras.engine.base_layer_utils.

13/13 [==============================] - 6s 463ms/step - loss: 1.8544 - accuracy: 0.2788 - val_loss: 1.4117 - val_accuracy: 0.2143
Epoch 2/1000
13/13 [==============================] - ETA: 0s - loss: 1.9903 - accuracy: 0.21 - ETA: 0s - loss: 1.5851 - accuracy: 0.28 - ETA: 0s - loss: 1.5318 - accuracy: 0.30 - 1s 111ms/step - loss: 1.5186 - accuracy: 0.3197 - val_loss: 1.3789 - val_accuracy: 0.2857
Epoch 3/1000
13/13 [==============================] - ETA: 0s - loss: 1.5903 - accuracy: 0.28 - ETA: 0s - loss: 1.6278 - accuracy: 0.31 - ETA: 0s - loss: 1.5771 - accuracy: 0.31 - 1s 99ms/step - loss: 1.5745 - accuracy: 0.3125 - val_loss: 1.3509 - val_accuracy: 0.2619
Epoch 4/1000
13/13 [==============================] - ETA: 0s - loss: 1.3113 - accuracy: 0.40 - ETA: 0s - loss: 1.5224 - accuracy: 0.36 - ETA: 0s - loss: 1.5658 - accuracy: 0.34 - 1s 102ms/step - loss: 1.5599 - accuracy: 0.3462 - val_loss: 1.3372 - val_accuracy: 0.2500
Epoch 5/1000
13/13 [==============================] - ETA: 0

Epoch 32/1000
13/13 [==============================] - ETA: 0s - loss: 0.9224 - accuracy: 0.65 - ETA: 0s - loss: 1.0875 - accuracy: 0.54 - ETA: 0s - loss: 0.9981 - accuracy: 0.56 - 2s 118ms/step - loss: 0.9908 - accuracy: 0.5745 - val_loss: 1.2220 - val_accuracy: 0.4762
Epoch 33/1000
13/13 [==============================] - ETA: 0s - loss: 0.8017 - accuracy: 0.71 - ETA: 0s - loss: 0.9262 - accuracy: 0.58 - ETA: 0s - loss: 0.8563 - accuracy: 0.62 - 1s 99ms/step - loss: 0.8563 - accuracy: 0.6298 - val_loss: 1.2019 - val_accuracy: 0.5000
Epoch 34/1000
13/13 [==============================] - ETA: 0s - loss: 0.9041 - accuracy: 0.65 - ETA: 0s - loss: 0.8591 - accuracy: 0.63 - ETA: 0s - loss: 0.8693 - accuracy: 0.62 - 1s 99ms/step - loss: 0.8693 - accuracy: 0.6250 - val_loss: 1.2174 - val_accuracy: 0.5000


KeyboardInterrupt: 

In [12]:
model_finder.evaluate(X_test, y_test)

907/907 [==============================] - ETA: 0s - loss: 1.1611 - accuracy: 0.59 - ETA: 6s - loss: 1.2300 - accuracy: 0.45 - ETA: 7s - loss: 1.1846 - accuracy: 0.44 - ETA: 7s - loss: 1.2379 - accuracy: 0.43 - ETA: 7s - loss: 1.2283 - accuracy: 0.43 - ETA: 7s - loss: 1.2071 - accuracy: 0.44 - ETA: 7s - loss: 1.2148 - accuracy: 0.45 - ETA: 7s - loss: 1.2470 - accuracy: 0.44 - ETA: 7s - loss: 1.2426 - accuracy: 0.44 - ETA: 7s - loss: 1.2349 - accuracy: 0.44 - ETA: 7s - loss: 1.2288 - accuracy: 0.44 - ETA: 7s - loss: 1.2302 - accuracy: 0.44 - ETA: 7s - loss: 1.2249 - accuracy: 0.44 - ETA: 7s - loss: 1.2197 - accuracy: 0.44 - ETA: 6s - loss: 1.2111 - accuracy: 0.44 - ETA: 6s - loss: 1.2088 - accuracy: 0.44 - ETA: 6s - loss: 1.2083 - accuracy: 0.44 - ETA: 6s - loss: 1.2039 - accuracy: 0.44 - ETA: 6s - loss: 1.2011 - accuracy: 0.44 - ETA: 6s - loss: 1.2016 - accuracy: 0.44 - ETA: 6s - loss: 1.2011 - accuracy: 0.44 - ETA: 6s - loss: 1.2019 - accuracy: 0.44 - ETA: 6s - loss: 1.2038 - accuracy

[1.1790481805801392, 0.4501379430294037]

In [13]:
end_time = time.time()

In [14]:
print("Time spent", end_time - start_time)

Time spent 3318.420706510544


In [15]:
joblib.dump(model_finder, f"autokeras_{train_size}.pkl")

['autokeras_500.pkl']